## All in One File Utility using Python
--------------------------------------------------------------------------------------------------------------------------------------------------- 

## Pdf Merger (standalone) 

In [ ]:
# Import necessary libraries
import PyPDF2
import os
import tkinter as tk
from tkinter import filedialog, simpledialog, ttk
from PIL import Image, ImageTk
from threading import Thread

# Define a class for the PDF Merger application
class PDFMergerApp:
    def __init__(self, root):
        """
        Constructor method for initializing the PDFMergerApp class.

        Parameters:
        - root: Tkinter root window
        """
        # Initialize the Tkinter root window
        self.root = root
        self.root.title("PDF Merger")
        self.root.geometry("500x400")

        # Configure the style for Tkinter elements
        self.style = ttk.Style()
        self.style.configure("TButton", padding=(10, 5), font=('Arial', 10))

        # Initialize attributes for PDF files, PDF icon, and merging progress
        self.pdf_files = []
        self.pdf_icon = self.load_pdf_icon()
        self.merging_progress = tk.StringVar()
        self.merging_progress.set("")

        # Set up the graphical user interface
        self.setup_gui()

    def load_pdf_icon(self):
        """
        Load and resize the PDF icon.

        Returns:
        - ImageTk.PhotoImage: PhotoImage object containing the resized PDF icon
        """
        icon_path = "path/to/pdf_icon.png"  # Replace with the actual path to your PDF icon
        if os.path.exists(icon_path):
            pdf_icon = Image.open(icon_path).resize((20, 20), Image.ANTIALIAS)
            return ImageTk.PhotoImage(pdf_icon)
        else:
            return None

    def setup_gui(self):
        """Set up the graphical user interface by configuring buttons, listbox, and result labels."""
        self.setup_buttons()
        self.setup_listbox()
        self.setup_result_label()

    def setup_buttons(self):
        """Set up buttons for merging PDFs, adding PDFs, and clearing the list."""
        merge_button = ttk.Button(self.root, text="Merge PDFs", command=self.on_merge_button_click)
        merge_button.pack(pady=10, padx=20, side=tk.TOP)

        add_button = ttk.Button(self.root, text="Add PDFs", command=self.on_add_button_click)
        add_button.pack(pady=5, padx=20, side=tk.TOP)

        clear_button = ttk.Button(self.root, text="Clear List", command=self.on_clear_button_click)
        clear_button.pack(pady=5, padx=20, side=tk.TOP)

    def setup_listbox(self):
        """Set up the listbox for displaying the added PDF files."""
        self.listbox = tk.Listbox(self.root, selectbackground="#a6a6a6", selectmode=tk.SINGLE)
        self.listbox.pack(expand=tk.YES, fill=tk.BOTH, padx=20, pady=5)

    def setup_result_label(self):
        """Set up result labels for displaying merging progress and the path of the merged PDF."""
        self.result_label = ttk.Label(self.root, text="", font=("Arial", 10, "italic"))
        self.result_label.pack(pady=5)

        self.progress_label = ttk.Label(self.root, textvariable=self.merging_progress, font=("Arial", 10, "italic"))
        self.progress_label.pack(pady=5)

    def on_merge_button_click(self):
        """Event handler for the 'Merge PDFs' button click."""
        # Open a file dialog to get the output path for the merged PDF
        output_pdf_path = filedialog.asksaveasfilename(
            defaultextension=".pdf", filetypes=[("PDF files", "*.pdf")]
        )

        # Check if there are PDF files to merge and an output path is provided
        if self.pdf_files and output_pdf_path:
            self.result_label.config(text="Merging in progress...")
            # Start a thread to merge PDFs asynchronously
            Thread(target=self.merge_pdfs, args=(output_pdf_path, self.pdf_files)).start()
        else:
            self.result_label.config(text="Merging canceled.")

    def merge_pdfs(self, output_path, input_paths):
        """
        Merge multiple PDFs into a single PDF.

        Parameters:
        - output_path: str, path for the merged PDF file
        - input_paths: list, paths of input PDF files to be merged
        """
        pdf_merger = PyPDF2.PdfMerger()

        total_files = len(input_paths)
        # Iterate through input PDF paths and append to the PDF merger
        for count, path in enumerate(input_paths, start=1):
            pdf_merger.append(path)
            progress_percentage = int((count / total_files) * 100)
            self.merging_progress.set(f'Merging: {progress_percentage}% ({count}/{total_files})')
            self.root.update_idletasks()  # Update the GUI

        # Write the merged PDF to the output path
        with open(output_path, 'wb') as output_file:
            pdf_merger.write(output_file)

        # Reset merging progress and update result label
        self.merging_progress.set("")
        self.result_label.config(text=f'Merged PDF saved at:\n{output_path}')

    def on_add_button_click(self):
        """Event handler for the 'Add PDFs' button click."""
        # Open a file dialog to select multiple PDF files
        files = filedialog.askopenfilenames(
            title="Select PDF files", filetypes=[("PDF files", "*.pdf")]
        )

        # If files are selected, extend the list of PDF files and update the listbox
        if files:
            self.pdf_files.extend(files)
            self.update_listbox()

    def on_clear_button_click(self):
        """Event handler for the 'Clear List' button click."""
        # Clear the list of PDF files and update the listbox
        self.pdf_files.clear()
        self.update_listbox()

    def update_listbox(self):
        """Update the listbox with the current list of PDF files."""
        self.listbox.delete(0, tk.END)
        for i, file_path in enumerate(self.pdf_files, start=1):
            file_name = os.path.basename(file_path)
            self.listbox.insert(tk.END, f"{i}. {file_name}", self.pdf_icon)

# Main function to create and run the Tkinter application
def main():
    root = tk.Tk()
    app = PDFMergerApp(root)
    root.mainloop()

# Entry point to execute the main function when the script is run
if __name__ == "__main__":
    main()

## Image to PDF

In [ ]:
# Import necessary modules
import os
import threading
from tkinter import Tk, filedialog, Button, Label, Listbox, Scrollbar, Canvas, VERTICAL
from PIL import Image, ImageTk
from reportlab.pdfgen import canvas

# Create a class for the Image to PDF Converter application
class ImageToPdfConverter:
    def __init__(self, root):
        # Initialize the class with the root window
        self.root = root
        self.root.title("Image to PDF Converter")

        # Initialize instance variables for image paths, output PDF path, and progress tracking
        self.images = []
        self.output_pdf_path = ""
        self.total_images = 0
        self.current_progress = 0

        # Create and configure GUI elements using Tkinter
        self.label = Label(root, text="Select Images:")
        self.label.grid(row=0, column=0, columnspan=2)

        # Listbox to display selected image filenames
        self.listbox = Listbox(root, selectmode="extended", height=10, width=50)
        self.listbox.grid(row=1, column=0, rowspan=6, padx=10, pady=5)

        # Scrollbar for the listbox
        self.scrollbar = Scrollbar(root, orient=VERTICAL)
        self.scrollbar.config(command=self.listbox.yview)
        self.scrollbar.grid(row=1, column=1, rowspan=6, sticky="ns")

        self.listbox.config(yscrollcommand=self.scrollbar.set)

        # Canvas to display thumbnail of the first selected image
        self.thumbnail_canvas = Canvas(root, height=100, width=150, bg="white")
        self.thumbnail_canvas.grid(row=1, column=2, rowspan=2, padx=10, pady=5)

        # Buttons for interacting with the application
        self.select_button = Button(root, text="Select Images", command=self.select_images)
        self.select_button.grid(row=3, column=2, pady=5)

        self.clear_button = Button(root, text="Clear List", command=self.clear_list)
        self.clear_button.grid(row=4, column=2, pady=5)

        self.convert_button = Button(root, text="Convert to PDF", command=self.convert_to_pdf)
        self.convert_button.grid(row=5, column=2, pady=5)

        # Label to display progress percentage during PDF creation
        self.progress_label = Label(root, text="")
        self.progress_label.grid(row=6, column=0, columnspan=2)

    # Method to allow the user to select images
    def select_images(self):
        selected_images = filedialog.askopenfilenames(
            title="Select Images",
            filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.gif;*.bmp")],
        )
        if selected_images:
            self.images.extend(selected_images)
            self.update_listbox()
            self.update_thumbnail()

    # Method to clear the list of selected images
    def clear_list(self):
        self.images.clear()
        self.update_listbox()
        self.update_thumbnail()

    # Method to initiate the PDF conversion process
    def convert_to_pdf(self):
        if not self.images:
            self.label.config(text="No images selected.")
            return

        # Ask the user for the output PDF file path
        self.output_pdf_path = filedialog.asksaveasfilename(
            title="Save PDF As",
            defaultextension=".pdf",
            filetypes=[("PDF files", "*.pdf")],
        )

        if not self.output_pdf_path:
            self.label.config(text="PDF conversion canceled.")
            return

        # Initialize progress variables and start a new thread for PDF creation
        self.total_images = len(self.images)
        self.current_progress = 0
        self.progress_label.config(text="")

        # Create a separate thread for PDF creation to prevent GUI freezing
        progress_thread = threading.Thread(target=self.create_pdf_thread)
        progress_thread.start()

    # Method to create PDF in a separate thread
    def create_pdf_thread(self):
        # Initialize a PDF canvas
        pdf = canvas.Canvas(self.output_pdf_path)

        # Iterate through selected images, add them to the PDF, and update progress
        for i, image_path in enumerate(self.images):
            image = Image.open(image_path)
            width, height = image.size
            pdf.setPageSize((width, height))
            pdf.drawInlineImage(image_path, 0, 0, width, height)

            # Update progress and percentage
            self.current_progress = i + 1
            progress_percentage = (self.current_progress / self.total_images) * 100
            self.progress_label.config(text=f"Progress: {progress_percentage:.2f}%")

            # Update the GUI to reflect progress
            self.root.update()

            # Add a new page for each image except the last one
            if i < len(self.images) - 1:
                pdf.showPage()

        # Save the PDF and update progress label
        pdf.save()
        self.progress_label.config(text=f"PDF saved at {self.output_pdf_path}.")
        self.current_progress = 0

    # Method to update the listbox with selected image filenames
    def update_listbox(self):
        self.listbox.delete(0, "end")
        for image_path in self.images:
            self.listbox.insert("end", os.path.basename(image_path))

    # Method to update the thumbnail display based on the first selected image
    def update_thumbnail(self):
        if self.images:
            first_image = Image.open(self.images[0])
            thumbnail = first_image.resize((150, 100), Image.ANTIALIAS)
            thumbnail = ImageTk.PhotoImage(thumbnail)
            self.thumbnail_canvas.create_image(75, 50, anchor="center", image=thumbnail)
            self.thumbnail_canvas.thumbnail = thumbnail
        else:
            # Clear the canvas if no images are selected
            self.thumbnail_canvas.delete("all")

# Main block: Create Tkinter root window and run the application
if __name__ == "__main__":
    root = Tk()
    app = ImageToPdfConverter(root)
    root.mainloop()

## PDF to Image

In [ ]:
import os
import threading
import shutil
from tkinter import Tk, filedialog, Button, Label, Listbox, Scrollbar, VERTICAL, messagebox
from PIL import Image
import fitz  # PyMuPDF

class PdfToImageConverter:
    def __init__(self, root):
        self.root = root
        self.root.title("PDF to Image Converter")

        self.pdf_path = ""
        self.image_paths = []
        self.total_pages = 0
        self.current_progress = 0

        self.label = Label(root, text="Select PDF:")
        self.label.grid(row=0, column=0, columnspan=2)

        self.listbox = Listbox(root, selectmode="extended", height=10, width=50)
        self.listbox.grid(row=1, column=0, rowspan=6, padx=10, pady=5)

        self.scrollbar = Scrollbar(root, orient=VERTICAL)
        self.scrollbar.config(command=self.listbox.yview)
        self.scrollbar.grid(row=1, column=1, rowspan=6, sticky="ns")

        self.listbox.config(yscrollcommand=self.scrollbar.set)

        self.select_button = Button(root, text="Select PDF", command=self.select_pdf)
        self.select_button.grid(row=3, column=2, pady=5)

        self.clear_button = Button(root, text="Clear List", command=self.clear_list)
        self.clear_button.grid(row=4, column=2, pady=5)

        self.zip_button = Button(root, text="Create Zip File", command=self.create_zip_file)
        self.zip_button.grid(row=5, column=2, pady=5)

        self.progress_label = Label(root, text="")
        self.progress_label.grid(row=6, column=0, columnspan=2)

    def select_pdf(self):
        self.pdf_path = filedialog.askopenfilename(
            title="Select PDF",
            filetypes=[("PDF Files", "*.pdf")],
        )
        if self.pdf_path:
            self.load_pdf_info()
            self.update_listbox()

    def clear_list(self):
        self.pdf_path = ""
        self.image_paths = []
        self.total_pages = 0
        self.current_progress = 0
        self.listbox.delete(0, "end")
        self.progress_label.config(text="")

    def create_zip_file(self):
        if not self.image_paths:
            messagebox.showerror("Error", "No images to zip. Please select a PDF file first.")
            return

        if not self.pdf_path:
            messagebox.showerror("Error", "No PDF selected.")
            return

        # Create a temporary folder for the images
        temp_folder = os.path.join(os.path.dirname(self.pdf_path), "temp_images")
        os.makedirs(temp_folder, exist_ok=True)

        # Save images to the temporary folder
        for i, image_path in enumerate(self.image_paths):
            page_number = i + 1
            output_filename = os.path.join(temp_folder, f"page_{page_number}.png")

            image = Image.open(image_path)
            image.save(output_filename)

        zip_filename = filedialog.asksaveasfilename(
            title="Save Zip As",
            defaultextension=".zip",
            filetypes=[("Zip files", "*.zip")],
        )

        if not zip_filename:
            messagebox.showinfo("Info", "Zip creation canceled.")
            return

        shutil.make_archive(os.path.splitext(zip_filename)[0], 'zip', temp_folder)

        # Delete temporary folder after creating the zip file
        shutil.rmtree(temp_folder)

        # Delete temporary images after creating the zip file
        self.delete_temp_images()

        self.progress_label.config(text=f"Zip file created at {zip_filename}.")

    def delete_temp_images(self):
        for image_path in self.image_paths:
            os.remove(image_path)

    def load_pdf_info(self):
        self.image_paths = []
        pdf_document = fitz.open(self.pdf_path)

        for page_number in range(pdf_document.page_count):
            page = pdf_document[page_number]
            image = page.get_pixmap()
            image_path = f"temp_page_{page_number + 1}.png"
            image.save(image_path)
            self.image_paths.append(image_path)

        pdf_document.close()

    def update_listbox(self):
        self.listbox.delete(0, "end")
        for image_path in self.image_paths:
            self.listbox.insert("end", os.path.basename(image_path))

if __name__ == "__main__":
    root = Tk()
    app = PdfToImageConverter(root)
    root.mainloop()


## PDF Compressor

## Image Compressor

## File Format conversion

## Image and Pdf Meta Data Editing

## PDF watermarking and Encryption Decryption